<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/main/Präsentationen/02-Extract-Transform-Load/Extract-Transform-Load-Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Einfache Transformationen

In [1]:
# Bibliotheken einmalig installieren

%pip install pyspark pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 57.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=03ef442277eb46d994aba43d87094f86e0f4e788b68ce3319963515146b28eda
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [7]:
!wget -O create_data_advacend.py https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data_advacend.py

--2022-05-31 18:36:21--  https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data_advacend.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11341 (11K) [text/plain]
Saving to: ‘create_data_advacend.py’

create_data_advacen 100%[===================>]  11.08K  --.-KB/s    in 0s      

2022-05-31 18:36:22 (79.9 MB/s) - ‘create_data_advacend.py’ saved [11341/11341]



In [8]:
import sqlite3
import pandas as pd
import pyspark.sql.functions as func 
import os
from pyspark.sql.functions import col

os.system("python3 create_data_advacend.py")

0

In [9]:
## (Py)Spark starten

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [10]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

#DataFrame für Datenbank erzeugen
df_fields= spark.createDataFrame(df_fields) 
# df_fields.printSchema()
df_fields.show()

(5, 'Barn Ground', 10)
(6, 'Bank', 10)
(7, 'Far Brossler', 20)
(8, 'Middle Broom', 20)
(9, 'Chalks', 25)
(10, 'Big Broom ', 60)


Dies ist das erzeugte Dataframe:

    field_id    field_name  area_in_sqm
0         5   Barn Ground           10
1         6          Bank           10
2         7  Far Brossler           20
3         8  Middle Broom           20
4         9        Chalks           25
5        10    Big Broom            60
+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
|       8|Middle Broom|         20|
|       9|      Chalks|         25|
|      10|  Big Broom |         60|
+--------+------------+-----------+



### Daten einlesen

In [11]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(10)


+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
+------------+-----+----+-----------------+-------+-------------+
only showing top 10 rows

+--------+-----+-------+-----------------+----+---

### Zeilen und Spalten abfragen

In [12]:
## Spalten abfragen:	Jeremy will sich Übersicht über generellen Wasserverbrauch der Gemüsesorten verschaffen -> dazu begutachtet er die Spalte 'Wasserverbrauch'  
df_veg.select("crop", "water_consumption").show()

+--------+-----------------+
|    crop|water_consumption|
+--------+-----------------+
|zucchini|                6|
|  radish|                4|
|zucchini|                5|
|  radish|                4|
|zucchini|                6|
|  radish|                5|
|zucchini|                7|
|  radish|                3|
|zucchini|                7|
|  radish|                6|
|zucchini|                9|
|  radish|                7|
|zucchini|               10|
|  radish|                8|
|zucchini|               13|
|  radish|                8|
|zucchini|               12|
|  radish|             null|
|zucchini|                5|
|zucchini|             null|
+--------+-----------------+



In [13]:
## Zeilen abfragen: Jeremy überlegt welche Pflanze er anbaut und schaut sich dafür die Zucchini genauer an

df_veg_lokal = df_veg.collect() # notwendig, falls man in einem verteilten Systen arbeitet
print(f"Informationen zu Zucchini in der 9. Zeile: {df_veg_lokal[8]}\n")

Informationen zu Zucchini in der 9. Zeile: Row(crop='zucchini', field=5, revenue=Row(net=0, tax=0), water_consumption=7, week=5, yield_per_sqm=5)



### Spalten hinzufügen und entfernen

In [14]:
## Spalten hinzufügen: Jeremy möchte die Einnahmen der Pflanzen pro Ertrag/m² ermitteln

df_fru_plus = df_fru.withColumn("lucrativeness", df_fru.revenue * df_fru.yield_per_sqm)
df_fru_plus.show()

+------------+-----+----+-----------------+-------+-------------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|lucrativeness|
+------------+-----+----+-----------------+-------+-------------+-------------+
|strawberries|    7|   1|               12|      0|            0|            0|
|strawberries|    7|   2|               10|      0|            0|            0|
|strawberries|    7|   3|               12|      0|            0|            0|
|strawberries|    7|   4|               14|      0|            0|            0|
|strawberries|    7|   5|               14|      0|            5|            0|
|strawberries|    7|   6|               18|     30|           10|          300|
|strawberries|    7|   1|               12|      0|            0|            0|
|strawberries|    7|   2|               10|      0|            0|            0|
|strawberries|    7|   3|               12|      0|            0|            0|
|strawberries|    7|   7|               

In [15]:
## Spalten entfernen:   Um einen klareren Blick auf die Daten zu erhalten, entfernt Jeremy alle Spalten
## die nicht über den Wasserverbrauch aussagen

df_veg_drop = df_veg.drop(*('field', 'revenue', 'week'))
df_fru_drop = df_fru.drop(*('field', 'revenue', 'week'))

df_fru_drop.show()
df_veg_drop.show()

+------------+-----------------+-------------+
|        crop|water_consumption|yield_per_sqm|
+------------+-----------------+-------------+
|strawberries|               12|            0|
|strawberries|               10|            0|
|strawberries|               12|            0|
|strawberries|               14|            0|
|strawberries|               14|            5|
|strawberries|               18|           10|
|strawberries|               12|            0|
|strawberries|               10|            0|
|strawberries|               12|            0|
|strawberries|               20|           25|
|strawberries|               26|           25|
|strawberries|               24|           25|
|strawberries|               10|           25|
|strawberries|             null|            0|
+------------+-----------------+-------------+

+--------+-----------------+-------------+
|    crop|water_consumption|yield_per_sqm|
+--------+-----------------+-------------+
|zucchini|              

### Dataframes konkatenieren

In [ ]:
#df_veg.show()

In [16]:
## Um die beiden Dataframes kombinieren zu können müssen sie in das gleiche Format gebracht werden.
## Achtung hier verändert sich der semantische Sinn der Spalte. Dies ist hier egal, weil wir sie weiter unten löschen.
df_veg_correct_revenue = df_veg.withColumn("revenue", func.column("revenue.net"))
#df_veg_correct_revenue.show()


## DataFrames Zusammenfügen: Jeremy überlegt ob er Gemüse oder Obst anbauen soll, 
## deshalb verschafft er sich eine Gesamtübersicht/Vergleich aller Obst- & Pflanzensorten: Obst- & Gemüsesorten konkatenieren

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberr

### Datenbereinigung 

In [17]:
## Dupliakte entfernen: Wie Jeremy gerade aufgefallen ist, enthällt die Tabelle noch Duplikate.
## (Siehe z.B. Woche 2 & 8 der Erdbeeren) Diese sollen nun entfernt werden.

df_fru_veg.dropDuplicates().show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|  10|               10|    100|           25|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   1|               12|      0|            0|
|    zucchini|    5|   0|                7|      5|            5|
|      rad

In [18]:
## Aufräumen:	Für Jeremy sind jetzt natürlich nur Einträge relevant, die einen Wasserverbrauch beinhalten.

df_fru_veg = df_fru_veg.dropna()
df_fru_veg.show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberr

### Gezielte Abfrage mit Bedingungen

In [19]:
## Wasserverbrauch anzeigen:    Jeremy möchte wissen, wie der gesamte Wasserverbrauch im letzten Jahr war. 
## Und wie der Wasserverbrauch bezogen auf die jeweilige Nutzpflanze ausfällt.

df_fru_veg.agg(func.sum("water_consumption")).show()

df_fru_veg.groupby('crop').sum("water_consumption").show()

df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(func.sum("water_consumption")).show()

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   319|
+----------------------+

+------------+----------------------+
|        crop|sum(water_consumption)|
+------------+----------------------+
|strawberries|                   194|
|    zucchini|                    80|
|      radish|                    45|
+------------+----------------------+

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   194|
+----------------------+



# Transformationen für Fortgeschrittene

Damit Jeremy weitere Informationen aus den Daten filtern kann, muss er sich erneut einen Überblick verschaffen. 
+ Unterstützen Sie Jeremy, in dem Sie die Tabellen der Felder und anzubauenden Lebensmittels darstellen.

In [20]:
df_fields.show(10)
df_fru_veg.show(30)

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
|       8|Middle Broom|         20|
|       9|      Chalks|         25|
|      10|  Big Broom |         60|
+--------+------------+-----------+

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|  

#### Spark Functions

Nutzen Sie Sparkfunktions um Jeremy den gesamte Wasserverbrauch im letzten Jahr darzustellen.

In [21]:
df_fru_veg.agg(func.sum("water_consumption")).show()

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   319|
+----------------------+



Anschließend möchte Jeremy noch wissen, wie der Wasserverbrauch für die unterschiedlichen Gewächsarten war. Geben Sie hierfür den Wasserbedarf der Erdberren an.

In [22]:
df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(func.sum("water_consumption")).collect()

[Row(sum(water_consumption)=194)]

#### Groupby() Bedingung

Für die kommenden Wochen ist mit erhöhten Niederschlägen zu rechnen. Um zu wissen, wann es sich lohnt, die Felder zu bewässern, will sich Jeremy einen Überblick über die durchschnittliche Wassermenge pro Woche verschaffen.

+ Helfen Sie Jeremy  mittels groupby() Befehl dabei und visualisieren Sie den durchschnittlichen Wasserverbrauch.

In [23]:
df_alles_was_angebaut_wird = df_fru_veg.groupBy("week").agg(func.avg("water_consumption"))
df_alles_was_angebaut_wird.show()


+----+----------------------+
|week|avg(water_consumption)|
+----+----------------------+
|   7|                  20.0|
|   6|                  18.0|
|   9|                  24.0|
|   5|                  14.0|
|   1|                  12.0|
|  10|                  10.0|
|   3|                  12.0|
|   8|                  17.5|
|   2|                  10.0|
|   4|                  14.0|
|   0|     5.454545454545454|
|  50|                   8.0|
|  32|                   8.0|
|  40|                  10.0|
|  16|                  10.0|
+----+----------------------+



Nachdem wir den Durchschnittlichen Wasserbrauch kennen, interessiert Jeremy wieviel Wasser jedes Lebensmittel pro Ernte verbraucht.  

In [24]:
gesamt_verbrauch =  df_fru_veg.groupBy("crop").agg(func.sum("water_consumption"))
gesamt_verbrauch.show()

+------------+----------------------+
|        crop|sum(water_consumption)|
+------------+----------------------+
|strawberries|                   194|
|    zucchini|                    80|
|      radish|                    45|
+------------+----------------------+



#### Join() Bedingung 

- Wie Sie in der Präsentation festgestellt haben, gibt es verschiedene Optionen zwei Tabellen miteinaner zu verbinden. 
+ In unserem Fall verwendenen wir nun den inner Join da wir alle Zeilen behalten möchten. 

Bisher musste Jereomy für die Übersicht der Felder und der einzelnen Lebensmittel immer zwei Tabellenen nutzen. 

+ Helfen Sie ihm nun die Arbeit mittels eines Joins zu erleichtern

In [25]:
gesamt_join = df_fru_veg.join(df_fields, df_fru_veg.field ==  df_fields.field_id,"inner")
gesamt_join.show(30)

+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|field_id|  field_name|area_in_sqm|
+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|    zucchini|    5|  40|                5|     10|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|               12|      9|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|               13|      8|           25|       5| Barn Ground|         10|
|    zucchini|    5|  16|               10|      7|           25|       5| Barn Ground|         10|
|    zucchini|    5|   8|                9|      6|           10|       5| Barn Ground|         10|
|    zucchini|    5|   0|                7|      5|            5|       5| Barn Ground|         10|
|    zucchini|    5|   0|                7|      4|            0|       5| Barn Ground|         10|


Um die Wirtschaftlichkeit unseres Unternehmen zu prüfen, möchte Jeremy sich den gesamten Ertag des jeweiligen Nahrungsmittel anzeigen. 

+ Mulitplizieren Sie hierfür die Spalten yield_per_sqm und_sqm area_in_sqm
+ Erzeugen Sie eine neue Spalte im DataFrame für das Ergebnis
+ Tipp: hierfür können Sie die Funcktion **col** der Sparkfunctions verwenden.

In [26]:

gesamt_join = gesamt_join.withColumn("yield_per_area", func.col("yield_per_sqm") * func.col("area_in_sqm"))
gesamt_join.show()

+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+--------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|field_id|  field_name|area_in_sqm|yield_per_area|
+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+--------------+
|    zucchini|    5|  40|                5|     10|           25|       5| Barn Ground|         10|           250|
|    zucchini|    5|  40|               12|      9|           25|       5| Barn Ground|         10|           250|
|    zucchini|    5|  40|               13|      8|           25|       5| Barn Ground|         10|           250|
|    zucchini|    5|  16|               10|      7|           25|       5| Barn Ground|         10|           250|
|    zucchini|    5|   8|                9|      6|           10|       5| Barn Ground|         10|           100|
|    zucchini|    5|   0|                7|      5|            5|       5| Barn 

In [27]:

gesamt_ertrag =  gesamt_join.groupBy("crop").agg(func.sum("revenue"), func.sum("yield_per_area"), func.sum("water_consumption"))
gesamt_ertrag = gesamt_ertrag.select(col("crop").alias("Crop"),col("sum(revenue)").alias("revenue") , col("sum(water_consumption)").alias("water_consumption"), col("sum(yield_per_area)").alias("yield_per_area"))
gesamt_ertrag.show()


+------------+-------+-----------------+--------------+
|        Crop|revenue|water_consumption|yield_per_area|
+------------+-------+-----------------+--------------+
|strawberries|    490|              194|          2300|
|    zucchini|     55|               80|          1150|
|      radish|     31|               45|          4000|
+------------+-------+-----------------+--------------+



Nachdem wir den Ertrag für die einzelnen Gemüse- oder Obstsorten kennen, müssen wir Jeremy bei der Ermittelung des Gewinn je Lebensmittel unter die Arme greifen.

+ Hierfür benötigen wir die zuvor ermittelten Liter je Sorte welche wir mit dem Wert von 0.2 Cent verrechnen. 
 
+ Anschließend müssen noch die Erwerbskosten für die  Pflanzensamen miteinbeziehen um unsere Produktionskosten zu erhalten.
  >Kosten für Samen :
  + Erdbeeren = 50 Cent
  + Rettich = 10 Cent
  + Zucchini = 15 Cent 


+ Geben Sie das Ergebnis absteigend an. 

Zunächst müssen wir noch bei dem Dataframe noch eine Spalte mit den Kosten für die Saatkosten hinzufügen.

In [28]:
from pyspark.sql.window import Window as W
rating = [15,50,10]
b = spark.createDataFrame([(l,) for l in rating], ['cost_of_seed'])
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.monotonically_increasing_id())
b = b.withColumn("idx", func.monotonically_increasing_id())

windowSpec = W.orderBy("idx")
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.row_number().over(windowSpec))
b = b.withColumn("idx", func.row_number().over(windowSpec))


gesamt_ertrag = gesamt_ertrag.join(b, gesamt_ertrag.idx == b.idx).drop("idx")
gesamt_ertrag.show()

+------------+-------+-----------------+--------------+------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|
+------------+-------+-----------------+--------------+------------+
|strawberries|    490|              194|          2300|          15|
|    zucchini|     55|               80|          1150|          50|
|      radish|     31|               45|          4000|          10|
+------------+-------+-----------------+--------------+------------+



In [29]:
gesamt_ertrag = gesamt_ertrag.withColumn("revenue_after_water_cost",col("revenue") - (col("water_consumption")* 0.02))
gesamt_ertrag = gesamt_ertrag.withColumn("netto_revenue",col("revenue_after_water_cost") - col("cost_of_seed"))
gesamt_ertrag = gesamt_ertrag.withColumn("revenue_after_water_cost", func.round(gesamt_ertrag["revenue_after_water_cost"], 2)).withColumn("netto_revenue", func.round(gesamt_ertrag["netto_revenue"], 2))
gesamt_ertrag.show()

+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|revenue_after_water_cost|netto_revenue|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|strawberries|    490|              194|          2300|          15|                  486.12|       471.12|
|    zucchini|     55|               80|          1150|          50|                    53.4|          3.4|
|      radish|     31|               45|          4000|          10|                    30.1|         20.1|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+



In [30]:
#revenue = Einnahme , yield = Ertrag
gesamt_ertrag.orderBy(col("netto_revenue").desc()).show()

+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|revenue_after_water_cost|netto_revenue|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|strawberries|    490|              194|          2300|          15|                  486.12|       471.12|
|      radish|     31|               45|          4000|          10|                    30.1|         20.1|
|    zucchini|     55|               80|          1150|          50|                    53.4|          3.4|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+



#### Lambda Funtionen

Um unsere Lebensmittel in unserem Hofladen gewinnbringend vertreiben können, müssen wir Jeremy bei der Preisfindung unterstützen. 

+ Ermitteln Sie hierfür den Preis pro Stück und erzeugen hierfür eine neue Spalte in dem Dataframe


In [31]:
rdd2 = gesamt_ertrag.rdd.map(lambda x: (x["Crop"], x["netto_revenue"]*1.19*0.01, x["netto_revenue"]*1.59*0.01)) 
cost_overview = rdd2.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview = cost_overview.withColumn("min_selling_price_per_piece", func.round(cost_overview["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview["lucrative_selling_price"], 2))
cost_overview.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()

+------------+---------------------------+-----------------------+--------+
|        Crop|min_selling_price_per_piece|lucrative_selling_price|currency|
+------------+---------------------------+-----------------------+--------+
|strawberries|                       5.61|                   7.49|       £|
|      radish|                       0.24|                   0.32|       £|
|    zucchini|                       0.04|                   0.05|       £|
+------------+---------------------------+-----------------------+--------+



Für die Nutzung von Lambda können Sie auch ganze Funktionen aufgerufen werden. Dies wird am folgenden Beispiel erklärt. 

In [32]:
# By Calling function
def calc_func(x):
    Crop = x.Crop
    Netto_Revenue = x.netto_revenue 
    min_selling_price_per_piece = Netto_Revenue*1.19*0.01
    lucrative_selling_price = Netto_Revenue*1.59*0.01
    return (Crop, min_selling_price_per_piece, lucrative_selling_price)

rdd_process=gesamt_ertrag.rdd.map(lambda x: calc_func(x))
cost_overview_with_func = rdd_process.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview_with_func = cost_overview_with_func.withColumn("min_selling_price_per_piece", func.round(cost_overview_with_func["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview_with_func["lucrative_selling_price"], 2))
cost_overview_with_func.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()


+------------+---------------------------+-----------------------+--------+
|        Crop|min_selling_price_per_piece|lucrative_selling_price|currency|
+------------+---------------------------+-----------------------+--------+
|strawberries|                       5.61|                   7.49|       £|
|      radish|                       0.24|                   0.32|       £|
|    zucchini|                       0.04|                   0.05|       £|
+------------+---------------------------+-----------------------+--------+

